<h4>Imports</h4>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers, regularizers

import pandas as pd

In [ ]:
# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001
initializer = tf.keras.initializers.GlorotUniform(seed=42)

<h4>Load Data</h4>

In [ ]:


train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

train_images = os.getcwd() + "/../data/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/../data/test_images/" + test_df.iloc[:, 0].values
train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values

def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    #Set Shape-Old Version
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE #-1
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset
    .shuffle(buffer_size=len(train_labels)) #Karıştır
    .map(read_image)                        #Her bir elemanı read_image fonksiyonundan geçir
    .batch(batch_size=BATCH_SIZE)           #Batchlere böl
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset
    .map(read_image)                        #Her bir elemanı read_image fonksiyonundan geçir
    .batch(batch_size=BATCH_SIZE)           #Batchlere böl
    .prefetch(buffer_size=AUTOTUNE)
) 
#TYPE: TensorSliceDataset

<h4>Create Model</h4>

In [ ]:
inputs = keras.Input(shape=(64, 64, 1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding="same",
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
    kernel_initializer=initializer
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY), kernel_initializer=initializer)(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(
    64, 3, activation="relu", kernel_regularizer=regularizers.l2(WEIGHT_DECAY), kernel_initializer=initializer
)(x)
x = layers.Conv2D(128, 3, activation="relu", kernel_initializer=initializer)(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu", kernel_initializer=initializer)(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation="relu", kernel_initializer=initializer)(x)
output1 = layers.Dense(10, activation="softmax", name="first_num", kernel_initializer=initializer)(x)
output2 = layers.Dense(10, activation="softmax", name="second_num", kernel_initializer=initializer)(x)

model = keras.Model(inputs=inputs, outputs=[output1, output2])

<h4>Compile Model</h4>

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

<h4>Train Model</h4>

In [ ]:
model.fit(train_dataset, epochs=5, verbose=2)

<h4>Test Model</h4>

In [ ]:
model.evaluate(test_dataset, verbose=2)